## Cài đặt môi trường và thư viện cần thiết

### Mục tiêu
Chuẩn bị môi trường làm việc cho bài toán Visual Question Answering (VQA), đảm bảo các thư viện xử lý ảnh, mô hình Deep Learning và đánh giá được cài đặt đúng phiên bản và tương thích với nhau.

### Giải thích
- Gỡ bỏ phiên bản Pillow hiện tại và cài đặt lại phiên bản `10.4.0` để tránh các lỗi liên quan đến đọc ảnh TIFF trong dataset PathVQA.
- Cài đặt và cập nhật các thư viện:
  - `datasets`: tải và xử lý dataset.
  - `transformers`: sử dụng các mô hình và tokenizer tiền huấn luyện.
  - `accelerate`: hỗ trợ huấn luyện trên GPU/TPU.
  - `evaluate`: tính toán các metric đánh giá.
- Cài đặt thêm:
  - `timm`: thư viện chứa nhiều mô hình thị giác (CNN, ViT).
  - `torchvision`: xử lý ảnh và các mô hình thị giác cơ bản.

### Input
- Không có dữ liệu đầu vào.

###


In [ ]:
!pip uninstall -y pillow
!pip install pillow==10.4.0
!pip install -U datasets transformers accelerate evaluate
!pip install timm torchvision

Found existing installation: pillow 10.4.0
Uninstalling pillow-10.4.0:
  Successfully uninstalled pillow-10.4.0
  Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.5 MB)


ok


## Import thư viện và thiết lập môi trường chạy

### Mục tiêu
Khởi tạo các thư viện cần thiết cho bài toán Visual Question Answering (VQA), đồng thời thiết lập seed và cấu hình thiết bị để đảm bảo khả năng tái lập kết quả trong quá trình huấn luyện và đánh giá mô hình.

### Giải thích
- Import các thư viện cơ bản cho xử lý dữ liệu, tính toán số và Deep Learning như `torch`, `numpy`, `random`.
- Sử dụng `torch.utils.data` để xây dựng dataset và dataloader.
- Thư viện `PIL` được dùng để đọc và xử lý hình ảnh.
- `datasets` hỗ trợ tải dataset PathVQA.
- `transformers` cung cấp tokenizer cho xử lý câu hỏi ngôn ngữ tự nhiên.
- `timm` cung cấp các mô hình thị giác tiền huấn luyện.
- Hàm `seed_all` được sử dụng để cố định seed cho Python, NumPy và PyTorch nhằm đảm bảo kết quả có thể tái lập.
- Thiết bị chạy được tự động chọn giữa GPU (CUDA) và CPU.

### Input
- Không có dữ liệu đầu vào.

### Output
- Môi trường làm việc được khởi tạo.
- Thiết bị chạy (`cuda` hoặc `cpu`) được xác định và in ra màn hình.


In [ ]:
import os, random, math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image

from datasets import load_dataset
from transformers import AutoTokenizer
import timm

def seed_all(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_all(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


## Tải và kiểm tra dataset PathVQA

### Mục tiêu
Tải dataset PathVQA từ Hugging Face Hub và kiểm tra nhanh cấu trúc dữ liệu để hiểu các trường thông tin có trong mỗi mẫu.

### Giải thích
- Sử dụng hàm `load_dataset` để tải dataset PathVQA.
- In ra thông tin tổng quát của dataset để quan sát các split (train, validation, test).
- Kiểm tra các key trong một mẫu dữ liệu thuộc tập train.
- In ra nội dung câu hỏi (`question`) và câu trả lời (`answer`) để hiểu định dạng dữ liệu văn bản.
- Kiểm tra kiểu dữ liệu của trường `image` để xác nhận dữ liệu hình ảnh được lưu dưới dạng phù hợp cho xử lý tiếp theo.

### Input
- Dataset PathVQA được tải từ Hugging Face Hub.

### Output
- Thông tin tổng quát của dataset.
- Danh sách các trường dữ liệu trong một mẫu.
- Ví dụ một câu hỏi và câu trả lời.
- Kiểu dữ liệu của trường hình ảnh.


In [ ]:
ds = load_dataset("flaviagiammarino/path-vqa")
print(ds)
print(ds["train"][0].keys())
print(ds["train"][0]["question"])
print(ds["train"][0]["answer"])
print(type(ds["train"][0]["image"]))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00007-f2d0e9ef9f022d(…):   0%|          | 0.00/42.8M [00:00<?, ?B/s]

data/train-00001-of-00007-47d8e0220bf6c9(…):   0%|          | 0.00/81.0M [00:00<?, ?B/s]

data/train-00002-of-00007-7fb5037c4c5da7(…):   0%|          | 0.00/104M [00:00<?, ?B/s]

...

data/test-00002-of-00003-162830843501982(…):   0%|          | 0.00/69.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19654 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6259 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6719 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'question', 'answer'],
        num_rows: 19654
    })
    validation: Dataset({
        features: ['image', 'question', 'answer'],
        num_rows: 6259
    })
    test: Dataset({
        features: ['image', 'question', 'answer'],
        num_rows: 6719
    })
})
dict_keys(['image', 'question', 'answer'])
where are liver stem cells (oval cells) located?
in the canals of hering
<class 'PIL.JpegImagePlugin.JpegImageFile'>


## Xây dựng từ điển câu trả lời (Answer Vocabulary)

### Mục tiêu
Tạo tập từ điển các câu trả lời phổ biến từ tập huấn luyện nhằm chuyển bài toán Visual Question Answering thành bài toán phân loại nhiều lớp.

### Giải thích
- Hàm `norm_ans` được sử dụng để chuẩn hóa câu trả lời bằng cách chuyển về chữ thường và loại bỏ khoảng trắng dư thừa.
- Sử dụng `Counter` để đếm tần suất xuất hiện của mỗi câu trả lời trong tập huấn luyện.
- Chỉ giữ lại các câu trả lời có tần suất xuất hiện lớn hơn hoặc bằng `min_freq` nhằm giảm nhiễu.
- Sắp xếp các câu trả lời theo tần suất giảm dần và chọn tối đa `top_k` câu trả lời phổ biến nhất.
- Thêm nhãn đặc biệt `<unk>` để biểu diễn các câu trả lời ngoài từ điển.
- Xây dựng các ánh xạ:
  - `a2i`: ánh xạ từ câu trả lời sang chỉ số.
  - `i2a`: ánh xạ từ chỉ số sang câu trả lời.
- In ra thông tin thống kê về kích thước từ điển và tỷ lệ bao phủ dữ liệu.

### Input
- `hf_split`: tập dữ liệu huấn luyện từ PathVQA.
- `top_k`: số lượng câu trả lời tối đa được giữ lại trong từ điển.
- `min_freq`: tần suất xuất hiện tối thiểu của câu trả lời.

### Output
- `vocab`: danh sách các câu trả lời trong từ điển.
- `a2i`: ánh xạ câu trả lời → chỉ số.
- `i2a`: ánh xạ chỉ số → câu trả lời.
- Thông tin thống kê về kích thước từ điển và tỷ lệ bao phủ.


In [ ]:
from collections import Counter

def norm_ans(a):
    return str(a).strip().lower()

def build_answer_vocab(hf_split, top_k=2000, min_freq=2):
    cnt = Counter()
    for a in hf_split["answer"]:
        cnt[norm_ans(a)] += 1

    items = [(a, c) for a, c in cnt.items() if c >= min_freq]
    items.sort(key=lambda x: x[1], reverse=True)

    vocab = ["<unk>"] + [a for a, _ in items[:max(0, top_k - 1)]]
    a2i = {a: i for i, a in enumerate(vocab)}
    i2a = {i: a for a, i in a2i.items()}

    kept = sum(cnt[a] for a in vocab[1:])
    total = sum(cnt.values())
    print("total answers:", len(cnt))
    print("vocab size:", len(vocab))
    print("coverage:", kept / total)

    return vocab, a2i, i2a

TOP_K = 2000
vocab, a2i, i2a = build_answer_vocab(ds["train"], top_k=TOP_K, min_freq=2)


total answers: 3225
vocab size: 783
coverage: 0.875699603134222


## Tokenizer và tiền xử lý dữ liệu đầu vào

### Mục tiêu
Chuẩn bị các thành phần cần thiết để xử lý dữ liệu văn bản (câu hỏi) và hình ảnh trước khi đưa vào mô hình Visual Question Answering.

### Giải thích
- Sử dụng `AutoTokenizer` với mô hình `distilbert-base-uncased` để mã hóa câu hỏi ngôn ngữ tự nhiên.
- Hàm `pil_to_tensor_rgb` thực hiện tiền xử lý hình ảnh:
  - Chuyển ảnh sang không gian màu RGB.
  - Thay đổi kích thước ảnh về kích thước cố định.
  - Chuẩn hóa giá trị pixel theo mean và standard deviation phổ biến trong các mô hình thị giác tiền huấn luyện.
  - Chuyển ảnh sang dạng tensor với định dạng `(C, H, W)`.
- Hàm `encode_answer` ánh xạ câu trả lời văn bản sang chỉ số tương ứng trong từ điển câu trả lời, các câu trả lời ngoài từ điển được gán về nhãn `<unk>`.

### Input
- `img`: ảnh đầu vào dạng PIL Image.
- `ans`: câu trả lời dạng văn bản.

### Output
- Tensor hình ảnh đã được chuẩn hóa.
- Chỉ số tương ứng của câu trả lời trong từ điển.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def pil_to_tensor_rgb(img, size=224):
    img = img.convert("RGB").resize((size, size), resample=Image.BICUBIC)
    x = np.asarray(img).astype(np.float32) / 255.0
    mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
    std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
    x = (x - mean) / std
    x = np.transpose(x, (2, 0, 1))
    return torch.from_numpy(x)

def encode_answer(ans):
    return a2i.get(norm_ans(ans), 0)


## Xây dựng Dataset cho PathVQA

### Mục tiêu
Định nghĩa lớp Dataset tùy chỉnh cho bài toán Visual Question Answering nhằm kết hợp xử lý hình ảnh, câu hỏi và câu trả lời trong một mẫu dữ liệu thống nhất.

### Giải thích
- Lớp `PathVQADataset` kế thừa từ `torch.utils.data.Dataset`.
- Dataset nhận một split từ Hugging Face (`hf_split`) làm nguồn dữ liệu.
- Trong phương thức `__getitem__`:
  - Hình ảnh được đọc từ dataset và tiền xử lý bằng hàm `pil_to_tensor_rgb`.
  - Câu hỏi được mã hóa bằng tokenizer với:
    - Cắt bớt câu hỏi dài (`truncation`)
    - Padding về độ dài cố định (`max_length`)
  - Câu trả lời được ánh xạ sang nhãn số thông qua từ điển câu trả lời.
- Dữ liệu trả về dưới dạng dictionary, phù hợp để sử dụng trực tiếp trong DataLoader và vòng lặp huấn luyện.

### Input
- `hf_split`: tập dữ liệu PathVQA (train / validation / test).
- `max_len`: độ dài tối đa của chuỗi token câu hỏi.
- `img_size`: kích thước ảnh đầu vào.

### Output
- Một mẫu dữ liệu gồm:
  - `pixel_values`: tensor hình ảnh đã tiền xử lý.
  - `input_ids`: tensor token của câu hỏi.
  - `attention_mask`: tensor attention mask cho câu hỏi.
  - `labels`: nhãn câu trả lời dạng số.


In [ ]:
class PathVQADataset(Dataset):
    def __init__(self, hf_split, max_len=32, img_size=224):
        self.hf = hf_split
        self.max_len = max_len
        self.img_size = img_size

    def __len__(self):
        return len(self.hf)

    def __getitem__(self, idx):
        item = self.hf[idx]

        img = item["image"]
        pixel = pil_to_tensor_rgb(img, self.img_size)

        tok = tokenizer(
            item["question"],
            truncation=True,
            padding="max_length",
            max_length=self.max_len,
            return_tensors="pt"
        )

        label = torch.tensor(
            encode_answer(item["answer"]),
            dtype=torch.long
        )

        return {
            "pixel_values": pixel,
            "input_ids": tok["input_ids"].squeeze(0),
            "attention_mask": tok["attention_mask"].squeeze(0),
            "labels": label
        }


## Xây dựng DataLoader cho huấn luyện và đánh giá

### Mục tiêu
Chuẩn bị các DataLoader cho tập huấn luyện, validation và test nhằm cung cấp dữ liệu theo batch cho quá trình huấn luyện và đánh giá mô hình VQA.

### Giải thích
- Hàm `collate_fn` được định nghĩa để gom các mẫu dữ liệu trong một batch:
  - Ghép tensor hình ảnh, câu hỏi và nhãn bằng `torch.stack`.
  - Đảm bảo dữ liệu đầu ra có kích thước đồng nhất.
- Khởi tạo các dataset:
  - `train_ds` cho tập huấn luyện.
  - `val_ds` cho tập validation.
  - `test_ds` cho tập test.
- Sử dụng `DataLoader` để tạo batch dữ liệu:
  - `shuffle=True` cho tập train nhằm tăng tính ngẫu nhiên.
  - `shuffle=False` cho validation và test.
  - `pin_memory=True` giúp tăng tốc truyền dữ liệu khi sử dụng GPU.
  - `num_workers` được thiết lập để tăng hiệu quả load dữ liệu song song.

### Input
- Dataset PathVQA đã được tiền xử lý (train, validation, test).
- Các tham số: `batch_size`, `num_workers`.

### Output
- `train_loader`: DataLoader cho huấn luyện.
- `val_loader`: DataLoader cho validation.
- `test_loader`: DataLoader cho đánh giá cuối cùng.


In [ ]:
def collate_fn(batch):
    return {
        "pixel_values": torch.stack([b["pixel_values"] for b in batch]),
        "input_ids": torch.stack([b["input_ids"] for b in batch]),
        "attention_mask": torch.stack([b["attention_mask"] for b in batch]),
        "labels": torch.stack([b["labels"] for b in batch]),
    }

train_ds = PathVQADataset(ds["train"])
val_ds   = PathVQADataset(ds["validation"])
test_ds  = PathVQADataset(ds["test"])

BATCH_SIZE = 32
NUM_WORKERS = 2

train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    collate_fn=collate_fn
)

val_loader = DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    collate_fn=collate_fn
)

test_loader = DataLoader(
    test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    collate_fn=collate_fn
)


## Kiểm tra batch dữ liệu từ DataLoader

### Mục tiêu
Xác nhận dữ liệu được load từ `DataLoader` có định dạng, kích thước và kiểu dữ liệu đúng trước khi đưa vào mô hình huấn luyện.

### Giải thích
- Lấy một batch dữ liệu từ `train_loader` bằng cách sử dụng `iter`.
- Duyệt qua từng thành phần trong batch và in ra:
  - Tên trường dữ liệu.
  - Kích thước tensor (`shape`).
  - Kiểu dữ liệu (`dtype`).
- Bước này giúp đảm bảo dữ liệu hình ảnh, câu hỏi và nhãn đã được chuẩn bị đúng theo thiết kế pipeline VQA.

### Input
- `train_loader`: DataLoader của tập huấn luyện.

### Output
- Thông tin về kích thước và kiểu dữ liệu của các tensor trong một batch:
  - `pixel_values`
  - `input_ids`
  - `attention_mask`
  - `labels`


In [ ]:
batch = next(iter(train_loader))
for k, v in batch.items():
    print(k, v.shape, v.dtype)


pixel_values torch.Size([32, 3, 224, 224]) torch.float32
input_ids torch.Size([32, 32]) torch.int64
attention_mask torch.Size([32, 32]) torch.int64
labels torch.Size([32]) torch.int64


## Định nghĩa mô hình Visual Question Answering (VQA)

### Mục tiêu
Xây dựng mô hình VQA baseline kết hợp đặc trưng hình ảnh và đặc trưng câu hỏi để dự đoán câu trả lời dưới dạng bài toán phân loại nhiều lớp.

### Giải thích
- Mô hình `VQAModel` kế thừa từ `torch.nn.Module`.
- Thành phần trích xuất đặc trưng hình ảnh:
  - Sử dụng Vision Transformer (`vit_base_patch16_224`) từ thư viện `timm`.
  - Mô hình được tiền huấn luyện và loại bỏ head phân loại để lấy vector đặc trưng ảnh.
- Thành phần mã hóa câu hỏi:
  - Sử dụng mô hình ngôn ngữ `DistilBERT` tiền huấn luyện.
  - Biểu diễn câu hỏi được lấy từ token `[CLS]` (vị trí đầu tiên của chuỗi).
- Đặc trưng hình ảnh và câu hỏi được nối (concatenation) để tạo biểu diễn đa phương thức.
- Biểu diễn kết hợp được đưa qua các tầng fully connected để dự đoán logits cho các lớp câu trả lời.

### Input
- `pixel_values`: tensor hình ảnh đầu vào có kích thước `(batch_size, 3, 224, 224)`.
- `input_ids`: tensor token của câu hỏi.
- `attention_mask`: tensor attention mask cho câu hỏi.

### Output
- `logits`: tensor đầu ra có kích thước `(batch_size, num_classes)`, biểu diễn phân phối dự đoán trên các lớp câu trả lời.


In [ ]:
class VQAModel(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.vision = timm.create_model(
            "vit_base_patch16_224",
            pretrained=True,
            num_classes=0
        )
        vision_dim = self.vision.num_features

        self.text = AutoModel.from_pretrained("distilbert-base-uncased")
        text_dim = self.text.config.hidden_size

        self.classifier = nn.Sequential(
            nn.Linear(vision_dim + text_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, num_classes)
        )

    def forward(self, pixel_values, input_ids, attention_mask):
        img_feat = self.vision(pixel_values)
        txt_out = self.text(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        txt_feat = txt_out.last_hidden_state[:, 0, :]
        fused = torch.cat([img_feat, txt_feat], dim=1)
        logits = self.classifier(fused)
        return logits


## Khởi tạo mô hình và kiểm tra forward pass

### Mục tiêu
Khởi tạo mô hình VQA với số lớp câu trả lời tương ứng và kiểm tra nhanh khả năng forward dữ liệu qua mô hình trước khi bước vào quá trình huấn luyện.

### Giải thích
- Xác định số lớp câu trả lời (`num_classes`) dựa trên kích thước từ điển câu trả lời.
- Khởi tạo mô hình `VQAModel` và chuyển mô hình sang thiết bị chạy (CPU hoặc GPU).
- Lấy một batch dữ liệu từ `train_loader` và chuyển toàn bộ tensor sang thiết bị tương ứng.
- Thực hiện forward pass của mô hình trong chế độ không tính gradient (`torch.no_grad`) để kiểm tra đầu ra.
- In ra kích thước của tensor logits nhằm xác nhận đầu ra của mô hình khớp với số lớp câu trả lời.

### Input
- `train_loader`: DataLoader của tập huấn luyện.
- Một batch dữ liệu gồm hình ảnh và câu hỏi.

### Output
- `logits`: tensor đầu ra của mô hình có kích thước `(batch_size, num_classes)`.


In [ ]:
from transformers import AutoModel
num_classes = len(vocab)
model = VQAModel(num_classes).to(device)

batch = next(iter(train_loader))
for k in batch:
    batch[k] = batch[k].to(device)

with torch.no_grad():
    logits = model(
        batch["pixel_values"],
        batch["input_ids"],
        batch["attention_mask"]
    )

print("logits shape:", logits.shape)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

logits shape: torch.Size([32, 783])


## Định nghĩa hàm huấn luyện và đánh giá mô hình

### Mục tiêu
Xây dựng các hàm huấn luyện và đánh giá mô hình VQA theo từng epoch, nhằm theo dõi loss và độ chính xác trong quá trình huấn luyện.

### Giải thích
- Hàm `train_one_epoch`:
  - Đặt mô hình ở chế độ huấn luyện (`model.train()`).
  - Lặp qua từng batch dữ liệu trong DataLoader.
  - Thực hiện forward pass, tính loss, lan truyền ngược (backpropagation) và cập nhật trọng số mô hình.
  - Tính toán loss trung bình và accuracy trên toàn bộ epoch.
- Hàm `evaluate`:
  - Đặt mô hình ở chế độ đánh giá (`model.eval()`).
  - Vô hiệu hóa tính gradient bằng decorator `@torch.no_grad()`.
  - Thực hiện forward pass trên tập validation hoặc test.
  - Tính loss trung bình và accuracy để đánh giá hiệu năng mô hình.

### Input
- `model`: mô hình VQA.
- `loader`: DataLoader của tập train, validation hoặc test.
- `optimizer`: thuật toán tối ưu hóa (chỉ dùng trong huấn luyện).
- `criterion`: hàm mất mát (Cross-Entropy).

### Output
- Loss trung bình trên toàn bộ dataset.
- Accuracy của mô hình trên tập dữ liệu tương ứng.


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for batch in loader:
        for k in batch:
            batch[k] = batch[k].to(device)

        optimizer.zero_grad()

        logits = model(
            batch["pixel_values"],
            batch["input_ids"],
            batch["attention_mask"]
        )

        loss = criterion(logits, batch["labels"])
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == batch["labels"]).sum().item()
        total += batch["labels"].size(0)

    return total_loss / len(loader), correct / total


@torch.no_grad()
def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    for batch in loader:
        for k in batch:
            batch[k] = batch[k].to(device)

        logits = model(
            batch["pixel_values"],
            batch["input_ids"],
            batch["attention_mask"]
        )

        loss = criterion(logits, batch["labels"])
        total_loss += loss.item()

        preds = logits.argmax(dim=1)
        correct += (preds == batch["labels"]).sum().item()
        total += batch["labels"].size(0)

    return total_loss / len(loader), correct / total


## Thiết lập huấn luyện và vòng lặp train – validation

### Mục tiêu
Thiết lập hàm mất mát, optimizer và thực hiện huấn luyện mô hình VQA trong nhiều epoch, đồng thời theo dõi kết quả trên tập validation.

### Giải thích
- Sử dụng hàm mất mát `CrossEntropyLoss` cho bài toán phân loại câu trả lời.
- Optimizer `AdamW` được sử dụng để tối ưu tham số mô hình, kết hợp với weight decay nhằm giảm overfitting.
- Số epoch huấn luyện được thiết lập cố định.
- Trong mỗi epoch:
  - Mô hình được huấn luyện trên tập train bằng hàm `train_one_epoch`.
  - Mô hình được đánh giá trên tập validation bằng hàm `evaluate`.
  - Loss và accuracy của cả train và validation được in ra để theo dõi quá trình huấn luyện.

### Input
- `train_loader`: DataLoader của tập huấn luyện.
- `val_loader`: DataLoader của tập validation.
- `criterion`: hàm mất mát Cross-Entropy.
- `optimizer`: AdamW optimizer.
- `EPOCHS`: số epoch huấn luyện.

### Output
- Log huấn luyện theo từng epoch, bao gồm:
  - Train loss và train accuracy.
  - Validation loss và validation accuracy.


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)

EPOCHS = 3

for epoch in range(EPOCHS):
    train_loss, train_acc = train_one_epoch(
        model, train_loader, optimizer, criterion
    )
    val_loss, val_acc = evaluate(
        model, val_loader, criterion
    )

    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"Train Loss {train_loss:.4f} Acc {train_acc:.4f} | "
        f"Val Loss {val_loss:.4f} Acc {val_acc:.4f}"
    )


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 1/3 | Train Loss 2.5441 Acc 0.4320 | Val Loss 2.4103 Acc 0.3529


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 2/3 | Train Loss 2.6470 Acc 0.3609 | Val Loss 2.2033 Acc 0.4422


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))
/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 3/3 | Train Loss 2.8122 Acc 0.3390 | Val Loss 3.0853 Acc 0.2489


## Đánh giá mô hình trên tập test

### Mục tiêu
Đánh giá hiệu năng cuối cùng của mô hình VQA trên tập test độc lập sau khi hoàn tất quá trình huấn luyện.

### Giải thích
- Sử dụng hàm `evaluate` để tính toán loss và accuracy của mô hình trên tập test.
- Mô hình được đặt ở chế độ đánh giá, không cập nhật trọng số trong quá trình inference.
- Kết quả test loss và test accuracy được in ra để làm chỉ số đánh giá cuối cùng của mô hình.

### Input
- `test_loader`: DataLoader của tập test.
- `criterion`: hàm mất mát Cross-Entropy.
- `model`: mô hình VQA đã được huấn luyện.

### Output
- `test_loss`: giá trị loss trên tập test.
- `test_acc`: độ chính xác của mô hình trên tập test.


In [ ]:
test_loss, test_acc = evaluate(model, test_loader, criterion)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Test Loss: 3.12280638558524
Test Accuracy: 0.24765590117577022


## Dự đoán câu trả lời cho một mẫu dữ liệu

### Mục tiêu
Thực hiện suy luận (inference) cho một mẫu dữ liệu cụ thể trong dataset PathVQA nhằm so sánh câu trả lời dự đoán của mô hình với nhãn thực tế.

### Giải thích
- Hàm `predict_one` được sử dụng để dự đoán câu trả lời cho một mẫu dữ liệu tại vị trí chỉ định.
- Mô hình được đặt ở chế độ đánh giá (`model.eval()`) và vô hiệu hóa gradient bằng decorator `@torch.no_grad()`.
- Hình ảnh được tiền xử lý và thêm chiều batch trước khi đưa vào mô hình.
- Câu hỏi được tokenizer mã hóa với độ dài cố định.
- Mô hình thực hiện forward pass để sinh logits.
- Nhãn `<unk>` được loại khỏi khả năng dự đoán bằng cách gán giá trị logit rất nhỏ.
- Chỉ số dự đoán được ánh xạ ngược sang câu trả lời dạng văn bản thông qua từ điển `i2a`.

### Input
- `model`: mô hình VQA đã được huấn luyện.
- `dataset`: dataset PathVQA.
- `idx`: chỉ số của mẫu dữ liệu cần dự đoán.

### Output
- Một dictionary gồm:
  - `question`: câu hỏi đầu vào.
  - `ground_truth`: câu trả lời đúng.
  - `prediction`: câu trả lời do mô hình dự đoán.


In [ ]:
@torch.no_grad()
def predict_one(model, dataset, idx):
    model.eval()

    item = dataset[idx]

    img = item["image"]
    pixel = pil_to_tensor_rgb(img).unsqueeze(0).to(device)

    tok = tokenizer(
        item["question"],
        truncation=True,
        padding="max_length",
        max_length=32,
        return_tensors="pt"
    )

    input_ids = tok["input_ids"].to(device)
    attention_mask = tok["attention_mask"].to(device)

    logits = model(pixel, input_ids, attention_mask)
    logits[0, 0] = -1e9   # cấm chọn <unk>
    pred_id = logits.argmax(dim=1).item()

    return {
        "question": item["question"],
        "ground_truth": item["answer"],
        "prediction": i2a[pred_id]
    }


## Dự đoán có điều kiện theo loại câu hỏi (Yes/No và Open-ended)

### Mục tiêu
Cải thiện bước suy luận (inference) bằng cách xử lý khác nhau giữa câu hỏi dạng yes/no và câu hỏi mở, giúp mô hình đưa ra câu trả lời phù hợp hơn với ngữ cảnh câu hỏi.

### Giải thích
- Hàm `is_yes_no_question` được sử dụng để xác định xem câu hỏi có thuộc dạng yes/no hay không dựa trên tiền tố của câu hỏi.
- Trong hàm `predict_one`:
  - Mô hình được đặt ở chế độ đánh giá và không tính gradient.
  - Hình ảnh và câu hỏi được tiền xử lý tương tự như trong quá trình huấn luyện.
  - Sau khi thu được logits từ mô hình:
    - Với câu hỏi yes/no:
      - Chỉ cho phép mô hình chọn giữa hai nhãn `yes` và `no`.
      - Các nhãn khác bị loại bỏ bằng cách gán logit rất nhỏ.
    - Với câu hỏi open-ended:
      - Loại bỏ các nhãn không phù hợp như `<unk>`, `yes`, `no`.
      - Chọn nhãn có logit lớn nhất trong các nhãn còn lại.
- Kết quả dự đoán được ánh xạ từ chỉ số sang câu trả lời dạng văn bản.

### Input
- `model`: mô hình VQA đã được huấn luyện.
- `dataset`: dataset PathVQA.
- `idx`: chỉ số của mẫu dữ liệu cần dự đoán.

### Output
- Một dictionary gồm:
  - `question`: câu hỏi đầu vào.
  - `ground_truth`: câu trả lời đúng.
  - `prediction`: câu trả lời do mô hình dự đoán.


In [ ]:
def is_yes_no_question(q):
    q = q.lower().strip()
    return q.startswith((
        "is ", "are ", "does ", "do ",
        "did ", "was ", "were ",
        "can ", "could ", "should ",
        "has ", "have ", "had "
    ))


@torch.no_grad()
def predict_one(model, dataset, idx):
    model.eval()
    item = dataset[idx]

    img = item["image"]
    pixel = pil_to_tensor_rgb(img).unsqueeze(0).to(device)

    tok = tokenizer(
        item["question"],
        truncation=True,
        padding="max_length",
        max_length=32,
        return_tensors="pt"
    )

    input_ids = tok["input_ids"].to(device)
    attention_mask = tok["attention_mask"].to(device)

    logits = model(pixel, input_ids, attention_mask)

    q = item["question"].lower()

    if is_yes_no_question(q):
        # CHỈ cho chọn yes / no
        yes_id = a2i.get("yes")
        no_id = a2i.get("no")

        mask = torch.full_like(logits, -1e9)
        mask[0, yes_id] = logits[0, yes_id]
        mask[0, no_id] = logits[0, no_id]

        pred_id = mask.argmax(dim=1).item()
    else:
        # OPEN-ENDED: CẤM <unk>, yes, no
        forbid = []
        if "<unk>" in a2i: forbid.append(a2i["<unk>"])
        if "yes" in a2i: forbid.append(a2i["yes"])
        if "no" in a2i: forbid.append(a2i["no"])

        for fid in forbid:
            logits[0, fid] = -1e9

        pred_id = logits.argmax(dim=1).item()

    return {
        "question": item["question"],
        "ground_truth": item["answer"],
        "prediction": i2a[pred_id]
    }

## Minh họa kết quả dự đoán trên tập test

### Mục tiêu
Minh họa trực quan kết quả dự đoán của mô hình VQA trên một số mẫu dữ liệu thuộc tập test, nhằm so sánh câu trả lời dự đoán với câu trả lời thực tế.

### Giải thích
- Lựa chọn một số chỉ số mẫu cố định trong tập test.
- Gọi hàm `predict_one` để thực hiện suy luận cho từng mẫu.
- In ra các thông tin:
  - Câu hỏi đầu vào.
  - Câu trả lời đúng (ground truth).
  - Câu trả lời do mô hình dự đoán.
- Các ví dụ này giúp đánh giá định tính khả năng hiểu hình ảnh và câu hỏi của mô hình.

### Input
- `model`: mô hình VQA đã được huấn luyện.
- `ds["test"]`: tập test của dataset PathVQA.
- Danh sách các chỉ số mẫu cần minh họa.

### Output
- Các cặp kết quả gồm câu hỏi, câu trả lời đúng và câu trả lời dự đoán được in ra màn hình.


In [ ]:
for idx in [0, 10, 50, 100]:
    out = predict_one(model, ds["test"], idx)
    print("Q:", out["question"])
    print("GT:", out["ground_truth"])
    print("Pred:", out["prediction"])
    print("-" * 50)


Q: what are positively charged, thus allowing the compaction of the negatively charged dna?
GT: the histone subunits
Pred: oral
--------------------------------------------------
Q: are numbers in the illustrations involved in pathogenesis of two main types of diabetes mellitus?
GT: no
Pred: yes
--------------------------------------------------
Q: does sectioned surface show lobulated mass with bluish cartilaginous hue infiltrating the soft tissues?
GT: yes
Pred: yes
--------------------------------------------------
Q: does lower part of the image show a separate encapsulated gelatinous mass?
GT: yes
Pred: yes
--------------------------------------------------
